In [7]:
pip install emoji

     |████████████████████████████████| 175 kB 4.0 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=53dbf3532530adb62f73674f836181271a0aece92f55c7c2d8823e18c7509b9c
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [1]:
# Base and Cleaning 
import pandas as pd
import numpy as np
from datetime import datetime
import sys

#Natural Language Processing (NLP)
import spacy  
from spacy.tokenizer import Tokenizer
import gensim
from gensim.corpora import Dictionary 
from gensim import models
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt


# settings for easier coding and inspection of data  
%matplotlib notebook
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.


In [2]:
import warnings
warnings.simplefilter('ignore')#"error", "ignore", "always", "default", "module"

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df_import = pd.read_csv('/content/drive/MyDrive/ColabNotebooks/EL/cyberbullying_tweets.csv')


In [5]:
df_import

,tweet_text,cyberbullying_type
0,"In other words #katandandre, your food was crapilicious! #mkr",not_cyberbullying
1,Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc,not_cyberbullying
2,@XochitlSuckkks a classy whore? Or more red velvet cupcakes?,not_cyberbullying
3,"@Jason_Gio meh. :P thanks for the heads up, but not too concerned about another angry dude on twitter.",not_cyberbullying
4,"@RudhoeEnglish This is an ISIS account pretending to be a Kurdish account. Like Islam, it is all lies.",not_cyberbullying
...,...,...
47687,"Black ppl aren't expected to do anything, depended on for anything. Yet free to participate, work, enjoy the freedom of humans all across this globe. If you waste your energy on the past you will be wasting it for building the future.",ethnicity
47688,Turner did not withhold his disappointment. Turner called the court an “abominable conclave of negro hating demons” (with one exception) who “issued another decree that colored men and women must be driven into Jim Crow cars whenever it suits the whim of any white community.”,ethnicity
47689,I swear to God. This dumb nigger bitch. I have got to bleach my hair reeeeeal fuckin' soon. D:&lt; FUCK.,ethnicity
47690,"Yea fuck you RT @therealexel: IF YOURE A NIGGER FUCKING UNFOLLOW ME, FUCKING DUMB NIGGERS.",ethnicity


In [6]:
temp = df_import.sample(5)

In [8]:
import emoji  # needs to be installed with pip
import regex # needs to be installed with pip
import re
import string
from collections import Counter

In [9]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    if not emoji_list==[] :
        print("",emoji_list,end="")
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text

def url_free_text(text):   
    '''
    Cleans text from urls
    '''
    text, _ = re.subn(r'http\S+', '', text)
    return text 

def url_free_text_s(text):  # as some https were missed out on by the previous function
    '''
    Cleans text from urls
    '''
    text, _ = re.subn(r'https\S+', '', text)
    return text  


# Apply the function above and get tweets free of emoji's
call_emoji_free = lambda x: give_emoji_free_text(x)

In [10]:
# Apply `call_emoji_free` which calls the function to remove all emoji's
# Apply 'url_free_text'which calls the function to remove all links

pd.options.mode.chained_assignment = None  # default='warn' turn off one the warning ... 

for c_b in df_import.cyberbullying_type.unique():
    temp_df = df_import[df_import.cyberbullying_type ==c_b]
    print(c_b,"'s tweets included the following emojis :")
    temp_df.tweet_text = temp_df.tweet_text.apply(call_emoji_free)
    print("\n")
    temp_df.tweet_text =temp_df.tweet_text.apply(url_free_text) 
    temp_df.tweet_text =temp_df.tweet_text.apply(url_free_text_s)
    df_import[df_import.cyberbullying_type ==c_b] = temp_df
    
df_prep=df_import # df_prep = result of preprocessing steps

not_cyberbullying 's tweets included the following emojis :


gender 's tweets included the following emojis :


religion 's tweets included the following emojis :


other_cyberbullying 's tweets included the following emojis :


age 's tweets included the following emojis :


ethnicity 's tweets included the following emojis :




In [11]:
# Load spacy
import spacy
import en_core_web_sm
#nlp = spacy.load('en_core_web_sm')
nlp = en_core_web_sm.load()

In [12]:
# Tokenizer
tokenizer = Tokenizer(nlp.vocab)

# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', 
                    "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']


# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)

# ALL_STOP_WORDS = spacy + gensim + wordcloud 
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)


In [13]:
n = 10
print("These are",n,"of our stopwords:\n")
for i in list(ALL_STOP_WORDS)[:n]:
    print(i)

These are 10 of our stopwords:

did
his
above
beside
themselves
we'll
detail
who's
everything
top


In [14]:
tokens = []

for doc in tokenizer.pipe(df_prep.tweet_text,batch_size=500):
    doc_tokens = []    
    for token in doc: 
        if token.text.lower() not in ALL_STOP_WORDS:  
            doc_tokens.append(token.text.lower())   
    tokens.append(doc_tokens)
    
# Makes tokens column
df_prep['tokens'] = tokens

In [15]:
df_prep.sample(2)

,tweet_text,cyberbullying_type,tokens
45149,"dumb bitches ! Go drink some bleach RT @tayyoung_: FUCK OBAMA, dumb ass nigger",ethnicity,"[dumb, bitches, !, drink, bleach, rt, @tayyoung_:, fuck, obama,, dumb, ass, nigger]"
44924,"Youre a nigger and you hate your life hmm, really? Lmfao Briana is a dumb cunt. Fuck you bitcchh.",ethnicity,"[youre, nigger, hate, life, hmm,, really?, lmfao, briana, dumb, cunt., fuck, bitcchh.]"


In [16]:

df_prep['tokens_back_to_text'] = [' '.join(map(str, l)) for l in df_prep['tokens']]


# to remove weird characters and valueless words (in sense of our analysis),
# we want to remove words with len<=2
# to keep interesting words with this length, we define the following list

list_useful_2char_word = ["uk","us","ad"] 


def get_lemmas(text):
    '''Used to lemmatize the processed tweets'''
    lemmas = []
    
    doc = nlp(text)
    for token in doc: 
        if ((token.is_stop == False) and (token.is_punct == False) and 
                (token.pos_!= 'PRON') and (token.is_digit== False) and 
                ((len(token)>2) or (token.text in list_useful_2char_word))): 
            lemmas.append(token.lemma_)
    
    return lemmas

In [17]:
# appllication of get_lemmas
df_prep['lemmas'] = df_prep['tokens_back_to_text'].apply(get_lemmas)


In [18]:
df_prep.sample(4) # visual inspection of lemmas

,tweet_text,cyberbullying_type,tokens,tokens_back_to_text,lemmas
20654,#StandwithSajid Indian muslims are supporting every anti nationals and they think we are idiots you gonna suffer your karma,religion,"[#standwithsajid, indian, muslims, supporting, anti, nationals, think, idiots, gonna, suffer, karma]",#standwithsajid indian muslims supporting anti nationals think idiots gonna suffer karma,"[standwithsajid, indian, muslims, support, anti, national, think, idiot, go, suffer, karma]"
3788,@mykitchenrules where is this place? It's beautiful! #MKR,not_cyberbullying,"[@mykitchenrules, place?, beautiful!, #mkr]",@mykitchenrules place? beautiful! #mkr,"[@mykitchenrule, place, beautiful, mkr]"
27519,ok time to write code bbl.,other_cyberbullying,"[ok, time, write, code, bbl.]",ok time write code bbl.,"[time, write, code, bbl]"
29592,"So, I guess it's really happened. The #mkr competition is running for eternity.",other_cyberbullying,"[so,, guess, happened., #mkr, competition, running, eternity.]","so, guess happened. #mkr competition running eternity.","[guess, happen, mkr, competition, run, eternity]"


In [19]:
## check if we still got tokens with length <= 2 that are not specified in out list
def too_short_lemmas(x):
    for single_lemma in x:
        if (len(single_lemma)<=2) and (single_lemma not in list_useful_2char_word):
                print(single_lemma,end=",")
                

def check_for_min_length_lemma(x):
    too_short=False
    for single_lemma in x:
        if (len(single_lemma)<=2) and (single_lemma not in list_useful_2char_word):
            too_short = True
    return too_short


print("There are still",
      len(df_prep[df_prep.lemmas.apply(lambda x:check_for_min_length_lemma(x))]),
         "tweets with lemmas of length <=2 that are not allowed.",
     "\nThese too short lemmas are:\n")

# check if these lemmas can be removed or if they include important information
print(df_prep.lemmas.apply(lambda x:too_short_lemmas(x)).values) 


# this happened because the token was for example going --> len(going)>2 but lemma = go

There are still 2445 tweets with lemmas of length <=2 that are not allowed. 
These too short lemmas are:

go,go,go,go,go,go,go,go,go,go,go,ff,go,go,ff,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,lb,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,10,go,go,go,go,ff,go,up,go,go,do,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,op,go,go,go,go,go,ff,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,do,go,go,go,go,go,go,go,go,do,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,ht,go,go,go,go,do,go,go,go,go,go,go,go,go,go,do,go,go,go,go,go,go,pc,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,do,go,go,go,go,go,go,go,go,go,go,go,go,no,go,go,go,go,go,go,go,go,go,go,go,go,go,go,re,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,go,aw,go,go,go,go,go,go,go,go,go,go,go,go,

In [20]:
def remove_too_short(x):
    for single_lemma in x:
        if (len(single_lemma)<=2) and (single_lemma not in list_useful_2char_word):
                x.remove(single_lemma)
    return x

df_prep.lemmas = df_prep.lemmas.apply(lambda x:remove_too_short(x))

print("There are still",
      len(df_prep[df_prep.lemmas.apply(lambda x:check_for_min_length_lemma(x))]),
         "tweets with lemmas of length <=2 that are not allowed.")
# because of the small amount the effort to catch these slips was rated to high

There are still 6 tweets with lemmas of length <=2 that are not allowed.


In [21]:
df_prep['lemmas_back_to_text'] = [' '.join(map(str,l)) for l in df_prep['lemmas']]

In [22]:
lemma_tokens_all = []
# Tokenizer function
def tokenize(text):
    """
    Parses a string into a list of semantic units (words)
    Args:
        text (str): The string that the function will tokenize.
    Returns:
        list: tokens parsed out
    """
    # first of the following does not seem to work - or is this bcs of string digits
    tokens = re.sub(r'[^a-zA-Z 0-9]', '', text) 
    tokens = re.sub(r'[%s]' % re.escape(string.punctuation), '', text) # Remove punctuation
    tokens = re.sub(r'\w*\d\w*', '', text) # Remove words containing numbers
    tokens = re.sub(r'@*!*\$*', '', text) # Remove @ ! $
    tokens = tokens.strip(',') 
    tokens = tokens.strip('?') 
    tokens = tokens.strip('!') 
    tokens = tokens.strip("'") 
    tokens = tokens.strip(".") 


    tokens = tokens.lower().split() # Make text lowercase and split it
                
    lemma_tokens_all.extend(tokens)
    return tokens



In [23]:
# Apply tokenizer to df_all_orgas_cut_off and parallel create a list for every cyberbullying_type
# witl all lemma tokens

temp_series = pd.Series()
for i in df_prep.cyberbullying_type.unique():
    temp_df_part= df_prep[df_prep.cyberbullying_type==i]
    exec(f'list_{i}=[]') # create list to store all cyberbullying_type specific lemma tokens
    temp_df_part['lemma_tokens'] = temp_df_part['lemmas_back_to_text'].apply(tokenize)
    exec(f'list_{i}.extend(lemma_tokens_all)')
    lemma_tokens_all=[]
    temp_series= temp_series.append(temp_df_part['lemma_tokens'])

df_prep['lemma_tokens'] =temp_series

In [24]:
df_prep.head(2)

,tweet_text,cyberbullying_type,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens
0,"In other words #katandandre, your food was crapilicious! #mkr",not_cyberbullying,"[words, #katandandre,, food, crapilicious!, #mkr]","words #katandandre, food crapilicious! #mkr","[word, katandandre, food, crapilicious, mkr]",word katandandre food crapilicious mkr,"[word, katandandre, food, crapilicious, mkr]"
1,Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc,not_cyberbullying,"[#aussietv, white?, #mkr, #theblock, #imacelebrityau, #today, #sunrise, #studio10, #neighbours, #wonderlandten, #etc]",#aussietv white? #mkr #theblock #imacelebrityau #today #sunrise #studio10 #neighbours #wonderlandten #etc,"[aussietv, white, mkr, theblock, imacelebrityau, today, sunrise, studio10, neighbour, wonderlandten, etc]",aussietv white mkr theblock imacelebrityau today sunrise studio10 neighbour wonderlandten etc,"[aussietv, white, mkr, theblock, imacelebrityau, today, sunrise, studio10, neighbour, wonderlandten, etc]"


In [25]:
# check how many tokens are left that do not comply to our length rule (if the number is small we keep them)
counter_01=0
for i in df_prep['lemma_tokens']:
    for k in i: 
        if (k.isnumeric()) or (len(k)<3) and (k  not in list_useful_2char_word):
            counter_01 +=1
            a=k
            print(k,"is of type",type(k),"and found in:",i,"\n")
print("There are only",counter_01,"lemma tokens left, that do not comply to our length rule")


1950 is of type <class 'str'> and found in: ['clippertea', 'target', 'demographic', '1950', 'businessman', 'apparently', 'fun', 'fact', 'draper', 'fucking', 'love', 'tea'] 

me is of type <class 'str'> and found in: ['verywhiteguy', 'me', 'twitter', 'search', 'look', 'block', 'manually', 'write'] 

na is of type <class 'str'> and found in: ['rgay', 'conversation', 'interest', 'have', 'write', 'possible', 'not', 'write', 'na'] 

th is of type <class 'str'> and found in: ['guiltyx', 'jillpantozzi', 'report', 'abuse', 'twitter', 'api', 'privacy', 'hack', 'support', 'freebsdgirl', 'th'] 

br is of type <class 'str'> and found in: ['br', 'yes', 'yes', 'money'] 

de is of type <class 'str'> and found in: ['zeynep', 'ugh', 'twitter', 'well', 'report', 'spam', 'block', 'single', 'menu', 'recombine', 'page', 'de'] 

vp is of type <class 'str'> and found in: ['mumtazceltik', 'whitehouse', 'vp', 'kobane', 'jesuischarlie', 'turkish', 'military', 'say', 'mit', 'ship', 'weapon', "qaeda'fehimtastekin

In [26]:
# Create a gensim dictionary out of ALL lemma tokens
gensim_dict = Dictionary(df_prep['lemma_tokens'])


In [27]:
def some_stats_on_gen_dict(gd):
    n=60
    #This module implements the concept of a Dictionary 
    # – a mapping between words and their integer ids.
    print("The dict was created by lookint at",gd.num_docs,
          "tweets \nIt contains",len(gd),
          "different lemma tokens \n\nHere are the first",n,"tweets:")
    for i in range(n):
        print (gd.get(i),end=", ")

some_stats_on_gen_dict(gensim_dict)



The dict was created by lookint at 47692 tweets 
It contains 53255 different lemma tokens 

Here are the first 60 tweets:
crapilicious, food, katandandre, mkr, word, aussietv, etc, imacelebrityau, neighbour, studio10, sunrise, theblock, today, white, wonderlandten, classy, cupcake, red, velvet, whore, xochitlsuckkks, angry, concerned, dude, head, jason_gio, meh, thank, twitter, account, isis, islam, kurdish, lie, pretend, rudhoeenglish, bad, existence, god, good, indifferent, prove, quickieleak, raja5aab, test, weird, yes, bukan, bully, itu, jauh, kaya, neraka, sekolah, tempat, bite, butt, hope, karma, kat, 

In [28]:
# create a corpus with all final lemma tokens for each cyberbullying_type
lemma_tokens_all = []
for sublist in df_prep.cyberbullying_type.unique():
    exec(f'lemma_tokens_all.append(list_{sublist})')

print(len(lemma_tokens_all)) 

corpus_all = [gensim_dict.doc2bow(d) for d in lemma_tokens_all]

# create work frequency list for all cyberbullying_type
word_frequencies = [[(gensim_dict[id], frequence) for id, 
                     frequence in couple] for couple in corpus_all]

6


In [29]:
word_frequencies_sorted = []
for tuples in word_frequencies:
    word_frequencies_sorted.append(sorted(tuples, key=lambda tup: tup[1], reverse = True))


In [30]:
k = []
for word_frequencies in word_frequencies_sorted:
    cumu_count = [0]
    counter = 1
    for i in word_frequencies:
        cumu_count.append(cumu_count[counter-1]+i[1])
        counter+=1

    num_element = 0
    
    for i in cumu_count:
        if i > 0.2 * max(cumu_count):
            k.append(num_element)
            break
        num_element += 1

print(k)

[65, 7, 21, 59, 3, 5]


In [31]:
c_b = 0
feature_list = []
for j in k:
    for i in range(0,j):
        feature_list.append(word_frequencies_sorted[c_b][i][0])
    c_b +=1
    

unique_feature_list = list(set(feature_list))
print("Number of words across all cyberbullying_type:",len(feature_list))
print("Number of unique words across all cyberbullying_type:",len(unique_feature_list))

Number of words across all cyberbullying_type: 160
Number of unique words across all cyberbullying_type: 101


In [32]:
unique_feature_list

['guy',
 'team',
 'muslims',
 'kat',
 'round',
 'school',
 'nigger',
 'female',
 'see',
 'mean',
 'daesh',
 'make',
 'stupid',
 'bullying',
 'cook',
 'lot',
 'right',
 'try',
 'rape',
 'start',
 'people',
 'twitter',
 'freebsdgirl',
 'woman',
 'man',
 'shit',
 'joke',
 'year',
 'love',
 'get',
 'know',
 'need',
 'feel',
 'dumb',
 'colin',
 'home',
 'use',
 'que',
 'tell',
 'mkr',
 'thing',
 'gay',
 'islamic',
 'terrorism',
 'kill',
 'terrorist',
 'time',
 'country',
 'isis',
 'fucking',
 'work',
 'ass',
 'quran',
 'new',
 'harassment',
 'hope',
 'class',
 'watch',
 'bully',
 'yeah',
 'think',
 'black',
 'talk',
 'christian',
 'fuck',
 'hate',
 'stop',
 'way',
 'yes',
 'feminazi',
 'pretty',
 'gamergate',
 'religion',
 'tweet',
 'say',
 'person',
 'idiot',
 'bad',
 'thank',
 'read',
 'look',
 'sure',
 'blameonenotall',
 'islam',
 'day',
 'high',
 'happen',
 'today',
 'good',
 'girl',
 'call',
 'andre',
 'radical',
 'mkr2015',
 'help',
 'muslim',
 'support',
 'bitch',
 'come',
 'dude',
 

In [33]:
dictOfWords = dict.fromkeys(unique_feature_list)
key_names = []
counter = 0
for key in dictOfWords:
    key_names.append(key)
    dictOfWords[key] = counter 
    counter+=1

lemma_list = list(df_prep.lemma_tokens)

len(lemma_list)
key_names.append('Not found')

In [34]:
feature_matrix = []
feature_size = len(unique_feature_list) + 1

lemma_list

for array in lemma_list:
    temp = [0]*feature_size
    for k in array:
        index = dictOfWords.get(k,feature_size-1)
        temp[index] = 1
    
    feature_matrix.append(temp)

print(feature_size)
print(len(feature_matrix))

102
47692


In [35]:
feature_df = pd.DataFrame(feature_matrix, index = df_prep.index, columns = key_names)
feature_df = feature_df.drop(['Not found'], axis=1)


In [36]:
feature_df.head(2)

,guy,team,muslims,kat,round,school,nigger,female,see,mean,...,andre,radical,mkr2015,help,muslim,support,bitch,come,dude,lol
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [37]:
combined_features = pd.concat([df_prep,feature_df],axis = 1)
combined_features.head(2)


,tweet_text,cyberbullying_type,tokens,tokens_back_to_text,lemmas,lemmas_back_to_text,lemma_tokens,guy,team,muslims,...,andre,radical,mkr2015,help,muslim,support,bitch,come,dude,lol
0,"In other words #katandandre, your food was crapilicious! #mkr",not_cyberbullying,"[words, #katandandre,, food, crapilicious!, #mkr]","words #katandandre, food crapilicious! #mkr","[word, katandandre, food, crapilicious, mkr]",word katandandre food crapilicious mkr,"[word, katandandre, food, crapilicious, mkr]",0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Why is #aussietv so white? #MKR #theblock #ImACelebrityAU #today #sunrise #studio10 #Neighbours #WonderlandTen #etc,not_cyberbullying,"[#aussietv, white?, #mkr, #theblock, #imacelebrityau, #today, #sunrise, #studio10, #neighbours, #wonderlandten, #etc]",#aussietv white? #mkr #theblock #imacelebrityau #today #sunrise #studio10 #neighbours #wonderlandten #etc,"[aussietv, white, mkr, theblock, imacelebrityau, today, sunrise, studio10, neighbour, wonderlandten, etc]",aussietv white mkr theblock imacelebrityau today sunrise studio10 neighbour wonderlandten etc,"[aussietv, white, mkr, theblock, imacelebrityau, today, sunrise, studio10, neighbour, wonderlandten, etc]",0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [38]:
# test whether the feature creation works
## entry number to test
test = 899
print(combined_features.iloc[test][6])
feature_df.iloc[test][feature_df.iloc[test]>0]


['halalflaws', 'greenlinerzjm', 'biebervalue', 'problem', 'force', 'wear', 'microbrain']


Series([], Name: 899, dtype: int64)

In [39]:
assert len(combined_features) == len(df_import) 

In [40]:
## save file from data frame to csv
combined_features.to_csv("bag of words_02.csv")